# agglomerative_CNN_clustering
- In [CNN_transfer_learning](./CNN_transfer_learning), we used a pre-trained convolutional neural network to generate a new feature set to better represent the galaxy images. These features were then compressed by a principal component analysis.
- In [kmeans_CNN_clustering](./kmeans_CNN_clustering.ipynb), we did a basic grouping into 10 clusters. Since there is not really a _good_ answer for how many clusters of galaxies we should have, so now lets try an algorithm that dictates the number of clusters.
- In [affinity_propagation_CNN_clustering](./affinity_propagation_CNN_clustering.ipynb), we used affinity propagation to automatically generate the clusters (and hence number and sizes). Using the higher dimensionality parameter space (i.e. output from CNN compress by the pca), we find that the process results in very precise clusters! The visual similarity is striking, however, is clustering based on postion angle (i.e. which direction the galaxy is pointing), colour, spiral (or other) structure and size/footprint of the galaxy in the image. Connecting these clusters is the difficult part, since many may simply be the same catagory/type of galaxy but viewed in a different way (position angle or footprint).
- To help aid this, we now look to hierarchical clustering (i.e. agglomerative - bottom-up) clustering to help define several levels of similarity (i.e, using ~175 clusters as a starting point).

In [2]:
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns

from sklearn.cluster import AgglomerativeClustering
import cluster_evaluate

/opt/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Loading set of features 
- Also weights in case we want to manipulate feature space to rank pca components that best describe the variance in the data
- Several feature representations were generated, so selecting combination of output features from CNN and the number of features these were compressed to by the pca.

In [3]:
pca_feat = 100
cnn_feat = 37632

features = np.load('./features/CNN-' + str(cnn_feat) + '_PCA-' + str(pca_feat) + '_features.npy')
weights = np.load('./features/CNN-' + str(cnn_feat) + '_PCA-' + str(pca_feat) + '_weights.npy')

plot_path = './cluster_plots/affinity_propagation/CNN-' + str(cnn_feat) + '_PCA-' + str(pca_feat) +'/'
plot_path

'./cluster_plots/affinity_propagation/CNN-37632_PCA-100/'

### Defining agglomerative model

In [ ]:
# define the model
agglo = AgglomerativeClustering(n_clusters=10, linkage='ward')
# fit the model
agglo_labels = agglo.fit_predict(compressed_features)